In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession .\
builder .\
appName("spark") .\
config("spark.sql.warehouse.dir","/user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [3]:
spark

In [4]:
from pyspark.sql.functions import *

In [5]:
from pyspark.sql import *

In [6]:
df1 = spark.read \
.format("csv") \
.option("header",True) \
.load("/public/trendytech/lendingclubproject/raw/loans_data_csv")

In [7]:
df1.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
56633077,b59d80da191f5b573...,3000.0,3000.0,36 months,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,36 months,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...
56473345,e5a140c0922b554b9...,20000.0,20000.0,36 months,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation
56463188,e12aefc548f750777...,11200.0,11200.0,60 months,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,60 months,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation


In [8]:
df2 = df1.withColumn("Time",current_timestamp())

In [9]:
df2.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
56633077,b59d80da191f5b573...,3000.0,3000.0,36 months,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-12 05:11:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,36 months,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-12 05:11:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,36 months,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-07-12 05:11:...
56463188,e12aefc548f750777...,11200.0,11200.0,60 months,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2025-07-12 05:11:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,60 months,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-07-12 05:11:...


In [10]:
df3 = df2.filter("loan_amnt is null")

In [11]:
columns = [
    "loan_id",
    "loan_amnt",
    "funded_amnt",
    "term",
    "int_rate",
    "installment",
    "issue_d",
    "loan_status",
    "purpose",
    "title"
]

In [12]:
df4 = df2.na.drop(subset=columns,)

In [13]:
df4.filter("loan_amnt is null")

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time


In [14]:
df4.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
145499677,a703357afc7be3fe3...,10000.0,10000.0,36 months,8.19,314.25,Dec-2018,Fully Paid,debt_consolidation,Debt consolidation,2025-07-12 05:11:...
144538467,a0c637c3df6764663...,5000.0,5000.0,36 months,15.02,173.38,Dec-2018,Current,other,Other,2025-07-12 05:11:...
145515405,63571114d3a96e5bc...,7500.0,7500.0,36 months,10.33,243.17,Dec-2018,Current,debt_consolidation,Debt consolidation,2025-07-12 05:11:...
145207340,4db14234c3f2f87c1...,20400.0,20400.0,60 months,16.14,497.61,Dec-2018,Current,home_improvement,Home improvement,2025-07-12 05:11:...
145467050,88a6f97ff3afc51b6...,12000.0,12000.0,36 months,7.02,370.64,Dec-2018,Current,credit_card,Credit card refin...,2025-07-12 05:11:...


In [15]:
df5 = df4.withColumn(
    "term",
    (regexp_replace(col("term"), " months", "").cast("int") / 12).cast("int")
)

In [16]:
df5.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
56633077,b59d80da191f5b573...,3000.0,3000.0,3,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-12 05:11:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-12 05:11:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-07-12 05:11:...
56463188,e12aefc548f750777...,11200.0,11200.0,5,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2025-07-12 05:11:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-07-12 05:11:...


In [17]:
df6 = df5.groupBy("purpose").count().sort("count",ascending= False)

In [24]:
df6

purpose,count
debt_consolidation,1263422
credit_card,511459
home_improvement,149141
other,138623
major_purchase,50050
medical,27247
small_business,24416
car,23815
vacation,15373
moving,15297


In [25]:
loan_purposes = [
    "debt_consolidation",
    "credit_card",
    "home_improvement",
    "other",
    "major_purchase",
    "medical",
    "small_business",
    "car",
    "vacation",
    "moving",
    "house",
    "wedding",
    "renewable_energy",
    "educational"
]

In [26]:
df5.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
91609139,f4d5593aeb85f0302...,2800.0,2800.0,3,10.49,91.0,Oct-2016,Fully Paid,car,Car financing,2025-07-12 05:11:...
91302887,2a6b46e98f2f63710...,8000.0,8000.0,3,13.49,271.45,Oct-2016,Fully Paid,credit_card,Credit card refin...,2025-07-12 05:11:...
91313928,2aaeecbb0cb90f9a5...,8000.0,8000.0,3,16.99,285.19,Oct-2016,Charged Off,moving,Moving and reloca...,2025-07-12 05:11:...
91343295,064393f5934a2eca2...,10125.0,10125.0,3,11.49,333.84,Oct-2016,Charged Off,credit_card,Credit card refin...,2025-07-12 05:11:...
91503678,5461aa5fb52d7380e...,18000.0,18000.0,3,6.99,555.71,Oct-2016,Fully Paid,debt_consolidation,Debt consolidation,2025-07-12 05:11:...


In [27]:
df7 = df5.withColumn("purpose",
                    
                when(col("purpose").isin(loan_purposes),col("purpose")).otherwise("other")
                      
                    )

In [31]:
df7.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
91609139,f4d5593aeb85f0302...,2800.0,2800.0,3,10.49,91.0,Oct-2016,Fully Paid,car,Car financing,2025-07-12 05:12:...
91302887,2a6b46e98f2f63710...,8000.0,8000.0,3,13.49,271.45,Oct-2016,Fully Paid,credit_card,Credit card refin...,2025-07-12 05:12:...
91313928,2aaeecbb0cb90f9a5...,8000.0,8000.0,3,16.99,285.19,Oct-2016,Charged Off,moving,Moving and reloca...,2025-07-12 05:12:...
91343295,064393f5934a2eca2...,10125.0,10125.0,3,11.49,333.84,Oct-2016,Charged Off,credit_card,Credit card refin...,2025-07-12 05:12:...
91503678,5461aa5fb52d7380e...,18000.0,18000.0,3,6.99,555.71,Oct-2016,Fully Paid,debt_consolidation,Debt consolidation,2025-07-12 05:12:...


In [ ]:
df7.groupBy("purpose").count().sort("count",ascending=False)